In [1]:
from mapping import ReportMapper
import numpy as np
import pandas as pd
import yaml
from post_mapping import RT30RT32PostMapper, MISSRSPostMapper, RMDPostMapper

%load_ext autoreload
%autoreload 2

In [5]:


from platform import python_version

print(python_version())

3.11.5


In [ ]:
path = r'../data/1.BOCS ARF 731 working 30092024 Dummy Data - 副本.xlsx'

na_values = ['-1.#IND', '1.#QNAN', '1.#IND', '-1.#QNAN', '#N/A N/A', '#N/A', 'N/A', 'n/a', '<NA>', '#NA', 'NULL', 'null', 'NaN', '-NaN', 'nan', '-nan', 'None', '']

schema_dict = {
        'deal_id':str,
        'ide_linkage_ref':str,
        'customer_nr':str,
        'facility_nr':str,
        'AP_CODE':str,
        'SUB CODE':str,
        'PROD_CODE':str,
        'ide_sourcesys_ref':str,
        'ide_linkage_type':str,
        'nationality':str
        }

df_facility_sql = pd.read_excel(
    path, 
    sheet_name='Facility', 
    dtype=schema_dict)[[
    'entity', 'facility_nr', 'customer_nr', 'customer_legal_name',
       'deal_type', 'currency', 'OBS amount', 'Counterparty type', 'Domicile',
       'source_system']]
    
    
df_rt30_sql = pd.read_excel(
        path,
        sheet_name='RT30_RT32',
        dtype=schema_dict,
        keep_default_na=False,
        na_values=na_values
    )[['ide_internal_party_ref', 'ide_linkage_ref', 'ide_linkage_type',
        'ide_sourcesys_ref', 'lot_type_fk', 'rca_accrint', 'rca_bookv',
        'rca_marketv', 'rca_prov_coll', 'rca_prov_indi', 'rv_coa',
        'rv_cpty_type', 'rv_rel_party_type', 'rv_currency_sub_group',
        'rv_mat_original', 'rv_mat_remaining.1', 'rca_deferred_fee',
        'rca_mtm_negative', 'rca_mtm_positive', 'entity', 'deal_id',
        'customer_nr.1', 'nationality', 'domicile', 'customer_legal_name',
        'deal_type.1', 'currency.1', 'tfi_id', 'source_table', 'value_date',
        'maturity_date', 'source_system']]
    
df_rt30_sql.columns = df_rt30_sql.columns.str.split('.').str[0]

df_rmd = pd.read_excel(
        path,
        sheet_name='RMD',
        dtype=schema_dict
    )[['ide_linkage_ref', '本期最终风险承担国家 Ultimate risk-bearing countries', '上期最终风险承担国']]


df_mis_srs = pd.read_excel(
        path,
        sheet_name='MIS-SRS',
        dtype=schema_dict
    )
df_mis_srs.columns = df_mis_srs.columns.str.strip()

df_mis_srs = df_mis_srs[['DATE', 'ENTITY', 'DBU OBU', 'STATE', 'ORGANISATION CODE', 'RESP CENTER', 'DEAL_ID', 'AP_CODE', 'AP_NAME', 'SUB CODE', 
       'PROD_CODE', 'A/L/E', 'MGT ITEM', 'SOURCE_SYSTEM', 'p1', 'p2', 'p3', 'p4', '管控类型', '管控部门', '前中后台', 'P DEPT', 'CURRENCY', 'Original Amount', 
       'AUD Equivalent', 'yearly_average_balance_ori', 'yearly_average_balance_aud', 'monthly_average_balance_ori', 'monthly_average_balance_aud', 'Last mth end YTD-ORI', 
       'Last mth end YTD-AUD', 'Last year end YTD-ORI', 'Last year end YTD-AUD', 'Last year mth end YTD-ORI', 'Last year mth end YTD-AUD','Budget']]



# Mapping RT30_RT32

In [ ]:
yaml_file = '../config/RT30_RT32.yaml'
mapper = ReportMapper(yaml_file)
df_rt30_mapped = mapper.map_data(df_rt30_sql)
rt30_logic = RT30RT32PostMapper(df_rt30_mapped, mapper)
df_rt30_output = rt30_logic.pre_process()

# Mapping MIS_SRS


In [ ]:
yaml_file = '../config/MIS_SRS.yaml'
mapper = ReportMapper(yaml_file)
df_mis_srs_mapped = mapper.map_data(df_mis_srs)
logic = MISSRSPostMapper(df_mis_srs_mapped)
df_mis_output = logic.process_all()


# Mapping RMD
- 在RT30页面的AG列(rv_coa)筛选BLS-AST-LOANS-PLAIN和BLS-AST-LOANS-TF，并在AC列(rca_bookv)筛选非0数据后，将筛选后的X列(ide_linkage_ref)Paste至RMD页面的A列

In [ ]:
# df_rmd = df_rt30_output[
#     (df_rt30_output['rv_coa'].isin(['BLS-AST-LOANS-PLAIN', 'BLS-AST-LOANS-TF'])) & 
#     (df_rt30_output['rca_bookv'] != 0)]\
#         [['ide_linkage_ref','customer_nr','Vis-à-vis counterparty sector','ide_sourcesys_ref','rca_bookv','customer_legal_name','nationality','domicile']]


In [ ]:
yaml_file = '../config/RMD.yaml'
mapper = ReportMapper(yaml_file, reuse_mappings=False)
df_rmd_mapped = mapper.map_data(df_rt30_output)

logic = RMDPostMapper(df_rmd_mapped)
df_rmd_output = logic.process_all()
mapper.add_mapping('RMD', df_rmd_output)



## Mapping Facility

In [ ]:
df_facility_sql

In [ ]:
yaml_file = '../config/Facility.yaml'
mapper = ReportMapper(yaml_file)
mapper.add_mapping('RMD', df_rmd_output)
df_facility_sql['AP code'] = df_facility_sql.deal_type.str[-4:]
df_facility_mapped = mapper.map_data(df_facility_sql)

# add risk in 
df_rt30_outout = rt30_logic.post_process()

In [ ]:
df_rt30_output[['Risk_in', 'Nationality','Vis-à-vis country', 'Risk Transfer']].to_clipboard(index=False)

In [ ]:
from post_mapping import FacilityPostMapper
logic = FacilityPostMapper(df_facility_mapped)
df_facility_output = logic.process_all()

# Validation


In [ ]:

from validation import AGvsGL

al_gl = AGvsGL(df_mis_output, df_rt30_output)
df_algl = al_gl.process_all()

In [ ]:

from validation import OBcheck

obcheck = OBcheck(df_mis_output, df_facility_output, df_731b)
df_ob_check = obcheck.process_all()

In [ ]:
df_ob_check

# Aggregation 731.1a and 731.1l


In [ ]:
from aggregation import ReportAggregator731AL

aggregator = ReportAggregator731AL(df_rt30_output)
df_731a = aggregator.generate_731a(df_algl)
df_731l = aggregator.generate_731l(df_algl)


In [ ]:
yaml_file = '../config/RT30_RT32.yaml'
mapper = ReportMapper(yaml_file)

In [ ]:
from aggregation import ReportAggregator731B
mapper.reload_mappings()
aggregator = ReportAggregator731B(ReportMapper, df_rt30_output)
df_731b = aggregator\
    .pre_ob_process(df_algl, df_facility_output)


In [ ]:
def add_top_header(df, header, start=2):
    multi_index = [('' if col not in df.columns[start:]
                    else header, col) for col in df.columns]
    df.columns = pd.MultiIndex.from_tuples(multi_index)

In [ ]:
result = df_731b.result
result['Part E']


for k, v in result.items():
    if k == "Part A":
        add_top_header(v, k, 3)
    else:
        add_top_header(v, k)

In [ ]:
df_731b.post_ob_process(df_ob_check)

In [ ]:
df_731b.[post_ob_process()

In [ ]:
multi_index = [('' if col not in a.columns[3:] else 'Part A', col) for col in a.columns]
a.columns = pd.MultiIndex.from_tuples(multi_index)

In [ ]:
def add_top_header(df, header, start=2):
    multi_index = [('' if col not in df.columns[start:] else header , col) for col in df.columns]
    df.columns = pd.MultiIndex.from_tuples(multi_index)

In [ ]:
add_top_header(a, "Part A", 3)
add_top_header(b, "Part B")

In [ ]:
pd.concat([a, b], axis=1)

In [ ]:
df_731b_e = df_731b_agg.get_part_e(df_facility_output)
aggregator.post_ob_process(df_ob_check, df_731b_e)

In [ ]:
df_ob_check

In [ ]:
df_731b_e

In [ ]:
df_731b_agg.get_part_b()

In [ ]:
cols = ['<=3 mths', '3m-12m', '1y-2y', '2y-5y', '5y+', 'Maturity bucket Adjustment',
       'Total', 'Unallocated']
upper_level = ['Part a' for i in ]
df.columns = pd.MultiIndex.from_tuples([(upper, col) if upper else (col, '') for upper, col in zip(upper_level, df.columns)])

In [ ]:
df_731b = df_731b_d.post_ob_process(df_ob_check)

In [ ]:
df_731b.columns

In [ ]:
['Vis-à-vis country', 'Vis-à-vis country name', 'GL adj',, , 
       'Derivative contracts', 'Guarantees', 'Credit commitments',
       'Guarantees adj']

In [ ]:
{
    'part a': ['<=3 mths','3m-12m', '1y-2y', '2y-5y', '5y+', 
               'Maturity bucket Adjustment','Total', 'Unallocated'],
    'part b': ['Banking institutions', 'Central banks','General government', 
               'Non-banking financial institutions', 'Non-financial corporations', 
               'Households and non-profits', 'Total international claims', 'Check'],
    'part c': ['Outward risk transfer','Inward risk transfer', 
               'Net transfer of risk to the ultimate borrower']
    'part d': []
}

In [ ]:
df_731b2 = df_731b.post_ob_process(df_ob_check)

In [ ]:
pd.DataFrame([None] * len(table1), columns=[" "])

In [ ]:
df_731b2

In [ ]:
df_mis_agg = df_mis_output[df_mis_output['G'] == 'G']\
            .groupby(['CCY'])\
            .agg(Sum_AUD_Equivalent=('AUD Equivalent', 'sum'))

In [ ]:
df_mis_agg['Sum_AUD_Equivalent'].sum(numeric_only=True)

In [ ]:
df_rt30_output[['Risk_in', 'Nationality','Vis-à-vis country', 'Risk Transfer']].to_clipboard(index=False)

In [ ]:
df_rt30_output[df_rt30_output.customer_nr.duplicated(keep=False)]

In [ ]:
# Create your sample tables
table1 = pd.DataFrame({
    "A": [1, 2, 3],
    "B": [4, 5, 6],
    "C": [7, 8, 9]
})

table2 = pd.DataFrame({
    "D": ["a", "b", "c"],
    "E": ["d", "e", "f"],
    "F": ["g", "h", "i"]
})

# Add a blank column to separate tables
blank_column = pd.DataFrame([None] * len(table1), columns=[" "])

# Combine tables into a single DataFrame
combined = pd.concat([table1, blank_column, table2], axis=1)

In [ ]:
combined.to_excel('test.xlsx', index=False)

In [ ]:
df_731b[
    df_731b['Vis-à-vis country'].notna() & df_731b['Vis-à-vis country'] != None
]

In [ ]:
rt30_agg2 = (df_rt30_output[
    (df_rt30_output.ALE == 'A') &
    (df_rt30_output.rv_mat_remaining > 1060000)]
        .groupby('Vis-à-vis country')['rca_deferred_fee']
        .sum()
        .reset_index()
)

In [ ]:
def add_maturity_bucket_adjustment(self):
        
        rt30_filtered = - self.rt30[
            (self.rt30['ALE'] == 'A') &
            (self.rt30['rv_mat_remaining'] > 1060000)
            ]
        
        rt30_agg = rt30_filtered\
            .groupby('Vis-à-vis country')\
            .agg(maturity_bucket_adjustment=('rca_bookv', 'sum'))
        
        self.output = self.output.merge(rt30_agg, on='Vis-à-vis country', how='left')
        
        return self

In [ ]:
4360084491 -41711

In [ ]:
base_mask = df_rt30_output['ALE'] == 'A'
condition = (
    (df_rt30_output['rv_mat_remaining_derived'] > 1000000) & 
    (df_rt30_output['rv_mat_remaining_derived'] <= 1003000)
)
# base_mask &= condition

rt30_filtered = df_rt30_output[base_mask]

In [ ]:
base_mask

In [ ]:
rt30_agg = (rt30_filtered
                    .groupby('Vis-à-vis country')
                    .agg({
                        'rca_bookv': 'sum',
                        'rca_accrint': 'sum',
                        'rca_deferred_fee': 'sum',
                    })
                    .reset_index()
)

sum_8754 = (
            rt30_filtered[rt30_filtered['deal_type_derived'] == '8754']
            .groupby('Vis-à-vis country')['rca_bookv']
            .sum()
            .reset_index()
            .rename(columns={'rca_bookv': '8754_bookv'})
)

In [ ]:
result = rt30_agg.merge(sum_8754, on='Vis-à-vis country', how='left').fillna({'8754_bookv':0})

In [ ]:
result['output'] = (
            result['rca_bookv'] - 
            2 * result['8754_bookv'] - 
            result['rca_deferred_fee'] +
            result['rca_accrint']
        )

In [ ]:
from validation import DervsGL

derivs = DervsGL(mapper, df_rt30_outout, df_mis_output)
rca_mtm = derivs.process_all()


In [ ]:
rca_mtm['1'].loc[len(rca_mtm['1'])] = ' '

In [ ]:
rca_mtm['1']

In [ ]:
for k, v in rca_mtm.items():
    v.loc[len(v)] = ''
    

In [ ]:
output = None 

for key, group in rca_mtm.groupby('group'):
    total_row = pd.DataFrame([{'group': 'TOTAL','Diff': group['Diff'].sum()}])
    blank_row = pd.DataFrame([{'group':None}])
    output = pd.concat([output, group, total_row, blank_row])
    

In [ ]:
output.to_excel('test.xlsx', index=False)

In [ ]:
pd.concat([group, total_row, pd.DataFrame([{'group':'1'}])])

In [ ]:
pd.concat([group, total_row, pd.DataFrame([{'核算码':None}])])

In [ ]:
total_row = pd.DataFrame([{'group': 'TOTAL','Diff': group['Diff'].sum()}])

pd.concat([group, total_row])

# Database connection


In [ ]:
from sqlalchemy import create_engine
from queries import RT30_RT32_SQL

with create_engine('mssql+pyodbc://DESKTOP-Q79EBHP/BOC?driver=ODBC+Driver+17+for+SQL+Server') as engine:
    
    df_rt30_rt32 = pd.read_sql(RT30_RT32_SQL, engine)
